<a href="https://colab.research.google.com/github/isegura/PLN-tema1/blob/main/practica_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica: Fundamentos de PLN con NLTK

**Asignatura:** Procesamiento de Lenguaje Natural con Aprendizaje Profundo (UC3M)

En esta práctica usarás **NLTK** para implementar tareas típicas de PLN (preprocesado, análisis léxico, n-gramas y recursos semánticos).  
El objetivo es entender *qué hacen* estas herramientas y cómo se integran en un flujo de trabajo.

✅ **Entrega sugerida**: sube este notebook completado (celdas con `TODO`).


## 0. Preparación del entorno
Ejecuta la instalación solo si NLTK no está disponible en tu entorno. En entornos con restricciones, consulta a la profesora.


In [1]:
# Si lo necesitas, descomenta:
# !pip install -q nltk

import nltk
import re
import math
from collections import Counter

print('NLTK version:', nltk.__version__)

NLTK version: 3.9.1


### Descarga de recursos
NLTK separa el código de los **recursos** (corpora, modelos, etc.). Descarga solo lo necesario.


In [21]:
downloads = [
    'punkt_tab',          # tokenización
    'stopwords',      # stopwords
    'wordnet',        # WordNet
    'omw-1.4',        # WordNet multilingüe
    'averaged_perceptron_tagger_eng',  # POS tagger (inglés)
    'maxent_ne_chunker_tab',
    'words',
]
for pkg in downloads:
    try:
        nltk.data.find(pkg)
    except LookupError:
        nltk.download(pkg)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


## 1. Dataset de juguete (texto en español)
Usaremos un pequeño conjunto de frases para practicar. Puedes ampliarlo con noticias, reseñas o textos de tu interés.


In [6]:
texts_es = [
    "Me encanta el PLN, pero a veces el preprocesamiento es tedioso.",
    "Hoy he leído un artículo interesante sobre modelos de lenguaje.",
    "La tokenización y la lematización son pasos comunes en PLN.",
    "Los transformers han cambiado muchas tareas de procesamiento del lenguaje.",
    "SpaCy y NLTK son herramientas útiles, aunque con objetivos distintos.",
]

print('Nº documentos:', len(texts_es))
print(texts_es[0])

Nº documentos: 5
Me encanta el PLN, pero a veces el preprocesamiento es tedioso.


## 2. Tokenización
La tokenización separa el texto en unidades (tokens). En NLTK hay tokenizadores por defecto.

### 2.1 Tokenización a nivel de frase y palabra
**Tarea:** tokeniza el primer texto en palabras y cuenta cuántos tokens produce.


In [8]:
from nltk.tokenize import word_tokenize, sent_tokenize
print(texts_es[0])
doc0 = texts_es[0]

# TODO: tokeniza doc0 en palabras usando word_tokenize
tokens0 = word_tokenize(doc0, language='spanish')
print(tokens0)
print('Nº tokens:', len(tokens0))

# Extra: tokenización por frases (si hay varias)
print('Frases:', sent_tokenize(doc0, language='spanish'))

Me encanta el PLN, pero a veces el preprocesamiento es tedioso.
['Me', 'encanta', 'el', 'PLN', ',', 'pero', 'a', 'veces', 'el', 'preprocesamiento', 'es', 'tedioso', '.']
Nº tokens: 13
Frases: ['Me encanta el PLN, pero a veces el preprocesamiento es tedioso.']


## 3. Normalización y limpieza
En pipelines reales suele normalizarse el texto: minúsculas, eliminación de puntuación, números, etc.

### 3.1 Función de preprocesado
**Tarea:** implementa `preprocess()` que:
1) pase a minúsculas, 2) elimine tokens que no sean letras (p. ej. puntuación), 3) elimine stopwords en español.


In [12]:
from nltk.corpus import stopwords

stop_es = set(stopwords.words('spanish'))

def preprocess(text):
    """Devuelve una lista de tokens limpios. Sin stopwords.
    Sin números. Sin signos de puntuación"""
    tokens = word_tokenize(text.lower(), language='spanish')
    # únicamente conservamos los tokens que sean palabras; eliminamos números y signos de puntuación, etc
    tokens = [t for t in tokens if re.fullmatch(r"[a-záéíóúüñ]+", t)]
    tokens = [t for t in tokens if t not in stop_es]
    return tokens

print(preprocess(texts_es[0]))

['encanta', 'pln', 'veces', 'preprocesamiento', 'tedioso']


## 4. Frecuencias y n-gramas
Vamos a calcular frecuencias y bigramas para ver patrones léxicos.

### 4.1 Frecuencia de tokens
**Tarea:** calcula el top-10 de tokens más frecuentes del corpus.


In [13]:
corpus_tokens = []
for t in texts_es:
    corpus_tokens.extend(preprocess(t))

freq = Counter(corpus_tokens)
print('Top-10 tokens:', freq.most_common(10))

Top-10 tokens: [('pln', 2), ('lenguaje', 2), ('encanta', 1), ('veces', 1), ('preprocesamiento', 1), ('tedioso', 1), ('hoy', 1), ('leído', 1), ('artículo', 1), ('interesante', 1)]


### 4.2 Bigrama más frecuente
**Tarea:** calcula los bigramas sobre los tokens del corpus y muestra los 10 más frecuentes.


In [14]:
from nltk import bigrams

bi = list(bigrams(corpus_tokens))
bi_freq = Counter(bi)
print('Top-10 bigramas:', bi_freq.most_common(10))

Top-10 bigramas: [(('encanta', 'pln'), 1), (('pln', 'veces'), 1), (('veces', 'preprocesamiento'), 1), (('preprocesamiento', 'tedioso'), 1), (('tedioso', 'hoy'), 1), (('hoy', 'leído'), 1), (('leído', 'artículo'), 1), (('artículo', 'interesante'), 1), (('interesante', 'modelos'), 1), (('modelos', 'lenguaje'), 1)]


## 5. Stemming (raíz) en español
El **stemming** reduce palabras a una raíz aproximada. No es una lematización: puede producir formas no lingüísticas.

**Tarea:** aplica un stemmer en español y compara tokens originales vs stems.


In [15]:
from nltk.stem.snowball import SnowballStemmer

stemmer_es = SnowballStemmer('spanish')

example = preprocess("Los modelos de lenguaje generan textos interesantes")
stems = [stemmer_es.stem(w) for w in example]
print('tokens:', example)
print('stems :', stems)

tokens: ['modelos', 'lenguaje', 'generan', 'textos', 'interesantes']
stems : ['model', 'lenguaj', 'gener', 'text', 'interes']


## 6. Semántica léxica con WordNet (ejemplo en inglés)
WordNet es un recurso semántico principalmente en inglés. Aquí lo usamos para explorar **sinónimos** y **relaciones**.

**Tarea:** para la palabra `bank`, muestra 2 synsets y sus definiciones. ¿Qué ambigüedad observas?


In [16]:
from nltk.corpus import wordnet as wn

word = 'bank'
synsets = wn.synsets(word)
print('Nº synsets:', len(synsets))
for s in synsets[:2]:
    print('-', s.name(), '->', s.definition())

# Extra: sinónimos del primer synset
lemmas = synsets[0].lemmas()
print('Lemas (sinónimos aproximados):', [l.name() for l in lemmas])

Nº synsets: 18
- bank.n.01 -> sloping land (especially the slope beside a body of water)
- depository_financial_institution.n.01 -> a financial institution that accepts deposits and channels the money into lending activities
Lemas (sinónimos aproximados): ['bank']


## 7. POS tagging (etiquetado morfosintáctico) (inglés)
NLTK incluye etiquetadores POS entrenados (principalmente para inglés). Esto es útil para entender el concepto.

**Tarea:** etiqueta el texto y calcula cuántos sustantivos (NN, NNS, NNP, NNPS) hay.


In [19]:
from nltk import pos_tag

text_en = "Transformers changed natural language processing and enabled new applications."
tokens_en = word_tokenize(text_en)
tags = pos_tag(tokens_en)
print(tags)

# TODO: cuenta sustantivos
noun_tags = {'NN', 'NNS', 'NNP', 'NNPS'}
n_nouns = sum(1 for _, tag in tags if tag in noun_tags)
print('Nº sustantivos:', n_nouns)

[('Transformers', 'NNS'), ('changed', 'VBD'), ('natural', 'JJ'), ('language', 'NN'), ('processing', 'NN'), ('and', 'CC'), ('enabled', 'VBD'), ('new', 'JJ'), ('applications', 'NNS'), ('.', '.')]
Nº sustantivos: 4


## 8. NER con chunking (inglés)
Ejemplo rápido de reconocimiento de entidades con el chunker de NLTK.

**Tarea:** ejecuta NER y observa qué entidades detecta.


In [22]:
from nltk import ne_chunk

text_en2 = "Barack Obama visited Madrid in 2016 and met Google researchers."
tokens2 = word_tokenize(text_en2)
tags2 = pos_tag(tokens2)
tree = ne_chunk(tags2)
print(tree)

# Extra: extraer entidades
entities = []
for subtree in tree:
    if hasattr(subtree, 'label'):
        ent = ' '.join([t for t, _ in subtree.leaves()])
        entities.append((ent, subtree.label()))
print('Entidades:', entities)

(S
  (PERSON Barack/NNP)
  (PERSON Obama/NNP)
  visited/VBD
  (GPE Madrid/NNP)
  in/IN
  2016/CD
  and/CC
  met/VBD
  (GPE Google/NNP)
  researchers/NNS
  ./.)
Entidades: [('Barack', 'PERSON'), ('Obama', 'PERSON'), ('Madrid', 'GPE'), ('Google', 'GPE')]


### Ejercicio 1: Clasificación simple basada en reglas
- Define una lista de palabras positivas y negativas (en español).
- Implementa una función que etiquete cada oración como `POS`, `NEG` o `NEU` según la frecuecia de palabras positivas y negativas. Si el número de palabras positivas y negativas es idéntico, el texto se etiqueta como 'NEU'.
- Evalúa sobre al menos 15 oraciones.

### Ejercicio 2: Comparativa stemming vs (pseudo)lematización
- Selecciona 20 palabras (inglés) y compara stemming vs lematización.
- Discute qué errores introduce cada enfoque.


---
### Preguntas de reflexión (para discutir)
1. ¿Qué errores puede introducir la tokenización?
2. ¿Cuándo es útil eliminar stopwords y cuándo puede ser perjudicial?
3. ¿Qué diferencia observas entre stemming y lematización?
4. ¿Por qué WordNet no es suficiente para capturar semántica contextual?
